In [18]:
import pandas as pd
import requests
from datetime import datetime
import matplotlib.pyplot as plt

In [19]:
filePath = '../Resources/Montgomery_Crash_Data.csv'
df = pd.read_csv(filePath)

In [ ]:
df.columns
df.info()

In [ ]:
df_dateTimeSeverity = df[['Crash Date/Time', 'Injury Severity']]
df_dateTimeSeverity.head()

In [ ]:
df_dateTimeSeverity.set_index('Crash Date/Time', inplace=True)
df_dateTimeSeverity.index = pd.to_datetime(df_dateTimeSeverity.index)
display(df_dateTimeSeverity.head())
display(df_dateTimeSeverity.tail())

In [ ]:
df_dateTimeSeverity = df_dateTimeSeverity.map(lambda x: x.capitalize() if isinstance(x, str) else x)
unique_severity = df_dateTimeSeverity['Injury Severity'].unique()
print(f"The unique_severity colums are as follows {df_dateTimeSeverity}")

In [ ]:
severity_count = df_dateTimeSeverity['Injury Severity'].value_counts()
display(severity_count)

In [ ]:
date_count = df_dateTimeSeverity.shape[0]
print(f"The total number of accidents is {date_count}")

In [ ]:
df_dateTimeSeverity = df_dateTimeSeverity.sort_values(by='Crash Date/Time')
display(df_dateTimeSeverity.head())
display(df_dateTimeSeverity.tail())

In [ ]:
df_dateTimeSeverity.dtypes

In [ ]:
df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])
df_dateTimeSeverity.dtypes

In [ ]:
plt.figure(figsize=(8, 5))
df_dateTimeSeverity['Injury Severity'].value_counts().plot(kind='bar')

plt.title('Injury Severity Count')
plt.xlabel('Injury Severity')
plt.ylabel('Count')
plt.grid(axis='y')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
df_no_apparent_injury = df_dateTimeSeverity[df_dateTimeSeverity['Injury Severity'] == 'No apparent injury']
df_suspected_minor_injury = df_dateTimeSeverity[df_dateTimeSeverity['Injury Severity'] == 'Suspected minor injury']
df_suspected_serious_injury = df_dateTimeSeverity[df_dateTimeSeverity['Injury Severity'] == 'Suspected serious injury']
df_fatal_injury = df_dateTimeSeverity[df_dateTimeSeverity['Injury Severity'] == 'Fatal injury']

injury_severity_list = [
    df_no_apparent_injury,
    df_suspected_minor_injury,
    df_suspected_serious_injury,
    df_fatal_injury
]

for i in injury_severity_list:
    display(i.head())

In [ ]:
for i in injury_severity_list:
    display(f"Total number {i["Injury Severity"].iloc[0]} {len(i)}")

In [ ]:

def makeNewDataFrame(df):
    df = df.copy()
    df['Time of Day'] = df.index.time
    df['HourOfAccident'] = pd.to_datetime(df['Time of Day'], format='%H:%M:%S').dt.hour
    return df

df_fatal_injury = makeNewDataFrame(df_fatal_injury)
df_suspected_serious_injury = makeNewDataFrame(df_suspected_serious_injury)
df_suspected_minor_injury = makeNewDataFrame(df_suspected_minor_injury)
df_no_apparent_injury = makeNewDataFrame(df_no_apparent_injury)

df_injury_list = [df_no_apparent_injury,
                df_suspected_minor_injury,
                df_suspected_serious_injury,
                df_fatal_injury]

for i in df_injury_list: 
    display(i.head())

In [33]:
def graph_injury_severity(df):
    plt.figure(figsize=(8, 5))
    plt.hist(df['HourOfAccident'], bins=24, color='blue', alpha=0.7)
    time_labels = [f'{str(i).zfill(2)}:00' for i in range(24)]
    plt.title('Time of Day')
    plt.grid(axis='y')
    plt.title(f'{i["Injury Severity"].iloc[0]} Time of Day')
    plt.xlabel('Time of Day')
    plt.ylabel('Accident Count')
    plt.xticks(range(24), time_labels, rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
for i in df_injury_list:
    graph_injury_severity(i)